# COPEX HiFiC 

## table of content:
- [Initialisation du projet](#Initialisation-du-projet)
- [Decompression](#Decompression)
- [Compressed vs Decompressed](#Compressed-vs-Decompressed)
- [Flooding USE CASE](#Flooding-USE-CASE)

# Initialisation du projet

## Imports

In [1]:
import requests # for downloading data
from requests.auth import HTTPBasicAuth  # for authentication
import os # for proper path handling
import torch # for ai
import time # to get downloading and execution time
from tqdm import tqdm  # to have progression bars
import panel as pn # graphic interface to do comparisons
from PIL import Image # for image management
import numpy as np # image to panel hadling
import matplotlib.pyplot as plt # for visualisation
import matplotlib.image as mpimg # for visualisation

## Test GPU

In [2]:
print("Torch version:",torch.__version__)
# Check if CUDA is available
if torch.cuda.is_available():
    # print cuda version
    print(f"Version de CUDA disponible: {torch.version.cuda}")
else:
    print("CUDA n'est pas disponible sur ce système.")

Torch version: 2.3.0.dev20240305+cu121
Version de CUDA disponible: 12.1


## Data downloading functions

In [3]:
def original_url_from_model_name(model_name):
    """return the url and final uncompressed filename"""
    return "https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/models/"+model_name, model_name

def download_model(model_name, output_folder, username, password,):
    # make sure output folder exist, create it if not
    os.makedirs(output_folder, exist_ok=True)

    # full url to donwload file
    url, model_final_name = original_url_from_model_name(model_name)
    print("url = ",url)
    # full output path for download
    download_path = os.path.join(output_folder, model_name)
    if os.path.exists(download_path) : 
        print(download_path,"already downloaded.")
        return
        
    # athentication
    auth = HTTPBasicAuth(username, password) 

    # Télécharger le fichier
    begin_time = time.time()
    #  get the total size for the progression bar
    file_size = int(requests.head(url, auth=auth).headers['Content-Length'])
     # download file
    try :
        with requests.get(url, auth=auth, stream=True) as response:
            with open(download_path, 'wb') as file, tqdm(
                    desc=f"Downloading [{model_name}]",
                    total=file_size,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    file.write(data)
                    bar.update(len(data))
    except :
        print(exception)
    end_time = time.time()
    download_and_save_time = end_time - begin_time
    print(download_path,"downloaded in ",download_and_save_time,"s.")
    return

In [4]:
def get_recon_path(granule_id, output_folder) :
    return os.path.join(output_folder,granule_id+"_compressed_RECON.png")
    
def compressed__high_url_from_granuleid(file_name):
    """return the url and final compressed filename"""
    return "https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/data_compressed/S1A_VH_L16_clamp_0_250_compressed_high/"+file_name+"_compressed.hfc",file_name+"_compressed.hfc" 

def compressed__low_url_from_granuleid(file_name):
    """return the url and final compressed filename"""    
    return "https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/data_compressed/S1A_VH_L16_clamp_0_250_compressed_low/"+file_name+"_compressed.hfc",file_name+"_compressed.hfc" 

def original_url_from_granuleid(file_name):
    """return the url and final uncompressed filename"""
    return "https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/data_uncompressed/S1A_VH_rgb_L16_clamp_0_250/"+file_name+".png",file_name+".png" 

def download_data(granule_id, output_folder, username, password, compressed=True, compression_model="low"):
    # make sure output folder exist, create it if not
    os.makedirs(output_folder, exist_ok=True)

    # full url to donwload file
    if compressed :  
        if compression_model == "low" : url,filename = compressed__low_url_from_granuleid(granule_id)
        if compression_model == "high" : url,filename = compressed__high_url_from_granuleid(granule_id)
    if not compressed : url,filename = original_url_from_granuleid(granule_id)
        
    # full output path for download
    download_path = os.path.join(output_folder, filename)
    if os.path.exists(download_path) : 
        print(download_path,"already downloaded.")
        return download_path,0
        
    # athentication
    auth = HTTPBasicAuth(username, password) 

    # Télécharger le fichier
    begin_time = time.time()
    #  get the total size for the progression bar
    file_size = int(requests.head(url, auth=auth).headers['Content-Length'])
     # download file
    try :
        with requests.get(url, auth=auth, stream=True) as response:
            with open(download_path, 'wb') as file, tqdm(
                    desc=f"Downloading [{filename}]",
                    total=file_size,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    file.write(data)
                    bar.update(len(data))
    except :
        print(exception)

    end_time = time.time()
    download_and_save_time = end_time - begin_time
    print(download_path,"downloaded in ",download_and_save_time,"s.")
    return download_path,download_and_save_time

In [7]:
granule_id = "S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12"
output_folder = os.path.join("output")
originals_folder = os.path.join(output_folder,"originals")
compressed_low_folder = os.path.join(output_folder,"compressed_low")
compressed_high_folder = os.path.join(output_folder,"compressed_high")
# usage example


original_path , download_time = download_data(granule_id, originals_folder ,username="esa",password="D25uXWjdjZEJ", compressed= False)
print("estimated time to download 200 original images : ",download_time*200,"s")

_ , download_time = download_data(granule_id, compressed_high_folder ,username="esa",password="D25uXWjdjZEJ", compressed= True, compression_model="high" )
print("estimated time to download 200 compressed images : ",download_time*200,"s")

_ , download_time = download_data(granule_id, compressed_low_folder ,username="esa",password="D25uXWjdjZEJ", compressed= True, compression_model="low" )
print("estimated time to download 200 compressed images : ",download_time*200,"s")




output\originals\S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12.png downloaded in  0.6129138469696045 s.
estimated time to download 200 original images :  122.5827693939209 s


output\compressed_high\S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed.hfc downloaded in  0.10335111618041992 s.
estimated time to download 200 compressed images :  20.670223236083984 s


output\compressed_low\S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed.hfc downloaded in  0.17021512985229492 s.
estimated time to download 200 compressed images :  34.043025970458984 s


In [8]:
print("list of files in ",output_folder," :")
print(os.listdir(output_folder))
print("list of files in ",originals_folder," :")
print(os.listdir(originals_folder))
print("list of files in ",compressed_low_folder," :")
print(os.listdir(compressed_low_folder))
print("list of files in ",compressed_high_folder," :")
print(os.listdir(compressed_high_folder))

list of files in  output  :
['compressed_high', 'compressed_low', 'decompressed_high', 'decompressed_low', 'flooding', 'originals']
list of files in  output\originals  :
['S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12.png']
list of files in  output\compressed_low  :
['logs', 'S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed.hfc']
list of files in  output\compressed_high  :
['logs', 'S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed.hfc']


# Decompression

In [9]:
models_folder = os.path.join("models")
decompressed_low_folder = os.path.join(output_folder,"decompressed_low")
decompressed_high_folder = os.path.join(output_folder,"decompressed_high")
recon_path_low = get_recon_path(granule_id, decompressed_low_folder)
recon_path_high = get_recon_path(granule_id, decompressed_high_folder)

## Using model Low

In [10]:
#downloading model low
model_low_name = "HIFIC_OID7_L16_10000_low.pt"
download_model(model_name = model_low_name,username="esa",password="D25uXWjdjZEJ",output_folder = models_folder)

url =  https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/models/HIFIC_OID7_L16_10000_low.pt
models\HIFIC_OID7_L16_10000_low.pt already downloaded.


In [11]:
model_low_path = os.path.join(models_folder, model_low_name)

In [12]:
!python ../high-fidelity-generative-compression-master/compress.py -i {compressed_low_folder} -ckpt {model_low_path} --save --output_dir {decompressed_low_folder} --only_decompress True

getting data from  output\compressed_low
only_decompress is true... getting the .hfc
decompress working on device [ cuda ] 
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\src\loss\perceptual_similarity\weights\v0.1\alex.pth
...[net-lin [alex]] initialized
...Done
image [ output\decompressed_low\S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed_RECON.png ] already decompressed, getting to the next


11_57_29 INFO - logger_setup: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\compress.py

100%|##########| 64/64 [00:00<00:00, 195.03it/s]
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
11_57_46 INFO - load_model

## Using model high

In [13]:
#downloading model high
model_high_name = "HIFIC_OID7_L16_10000_high.pt"
download_model(model_name = model_high_name,username="esa",password="D25uXWjdjZEJ",output_folder = models_folder)

url =  https://www.visioterra.fr/telechargement/P382_ESRIN_COPEX-DCC/models/HIFIC_OID7_L16_10000_high.pt
models\HIFIC_OID7_L16_10000_high.pt already downloaded.


In [14]:
model_high_path = os.path.join(models_folder, model_high_name)

In [15]:
!python ../high-fidelity-generative-compression-master/compress.py -i {compressed_high_folder} -ckpt {model_high_path} --save --output_dir {decompressed_high_folder} --only_decompress True

getting data from  output\compressed_high
only_decompress is true... getting the .hfc
decompress working on device [ cuda ] 
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\src\loss\perceptual_similarity\weights\v0.1\alex.pth
...[net-lin [alex]] initialized
...Done
image [ output\decompressed_high\S1A_IW_GRDH_1SDV_20160511T043458_20160511T043523_011206_010EF8_77F4_12_compressed_RECON.png ] already decompressed, getting to the next


11_57_55 INFO - logger_setup: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\compress.py

100%|##########| 64/64 [00:00<00:00, 267.11it/s]
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
11_58_10 INFO - load_model

# Compressed vs Decompressed

Settings for panel

In [16]:
pn.extension()
pn.config.theme = 'dark'
loading_text = pn.pane.Str(
    'Processing request...',
    styles={'font-size': '12pt'}
)
loading_gif = pn.pane.GIF('https://upload.wikimedia.org/wikipedia/commons/d/de/Ajax-loader.gif')
loading = pn.Row(loading_gif, loading_text)

resizing function (keep image ratio)

In [17]:
def resize_and_save(image_path, width):
    # open image
    image = Image.open(image_path)

    # Calculate new height while maintaining aspect ratio
    orig_width, orig_height = image.size
    height = int((width / orig_width) * orig_height)

    # Resize image
    resized_image = image.resize((width, height))

    # Save resized image with "_tmp" added to file name
    output_path = os.path.splitext(image_path)[0] + "_tmp" + os.path.splitext(image_path)[1]
    resized_image.save(output_path)

    # Supprimer l'image originale
    # os.remove(image_path)

    #print("L'image a été redimensionnée et enregistrée avec succès sous:", output_path)
    return output_path


display a comparison between original and decompressed

## Comparison with low model

In [21]:
display_widget = pn.Row(loading_gif, pn.pane.Str('Processing request...',styles={'font-size': '12pt'}))
display(display_widget) # loading screen while preparing next widget
#resizing width to get better viewing experience
img_width = 1500

ori_resized = resize_and_save(original_path, width=img_width)
rec_resized = resize_and_save(recon_path_low, width=img_width)

display_widget.clear();
#display_widget.append(pn.Swipe(original_path, recon_path_low,slider_width=3, slider_color ="#00fff2"))
display_widget.append(pn.Swipe(ori_resized, rec_resized,slider_width=3, slider_color ="#00fff2"))

Row
    [0] GIF(str)
    [1] Str(str, styles={'font-size': '12pt'})

In [28]:
#removing tmp resized images
os.remove(ori_resized)
os.remove(rec_resized)

## Comparison with high model

In [19]:
display_widget = pn.Row(loading_gif, pn.pane.Str('Processing request...',styles={'font-size': '12pt'}))
display(display_widget) # loading screen while preparing next widget
#resizing width to get better viewing experience
img_width = 1500

ori_resized = resize_and_save(original_path, width=img_width)
rec_resized = resize_and_save(recon_path_high, width=img_width)

display_widget.clear();
#display_widget.append(pn.Swipe(original_path, recon_path_high,slider_width=3, slider_color ="#00fff2"))
display_widget.append(pn.Swipe(ori_resized, rec_resized,slider_width=3, slider_color ="#00fff2"))


Row
    [0] GIF(str)
    [1] Str(str, styles={'font-size': '12pt'})

In [19]:
#removing tmp resized images
os.remove(ori_resized)
os.remove(rec_resized)

## Comparison between low and high reconstructions

In [20]:
display_widget = pn.Row(loading_gif, pn.pane.Str('Processing request...',styles={'font-size': '12pt'}))
display(display_widget) # loading screen while preparing next widget
#resizing width to get better viewing experience
img_width = 1500

ori_resized = resize_and_save(recon_path_low, width=img_width)
rec_resized = resize_and_save(recon_path_high, width=img_width)

display_widget.clear();
#display_widget.append(pn.Swipe(recon_path_low, recon_path_high,slider_width=3, slider_color ="#00fff2"))
display_widget.append(pn.Swipe(ori_resized, rec_resized,slider_width=3, slider_color ="#00fff2"))

Row
    [0] GIF(str)
    [1] Str(str, styles={'font-size': '12pt'})

In [21]:
#removing tmp resized images
os.remove(ori_resized)
os.remove(rec_resized)

# Flooding USE CASE

In [22]:
granule_id_dry = "S1A_IW_GRDH_1SDV_20210906T043536_20210906T043601_039556_04ACE3_18DD_12"
granule_id_flooded = "S1A_IW_GRDH_1SDV_20211129T043536_20211129T043601_040781_04D736_2E2A_12"
use_case_folder = os.path.join(output_folder,"flooding")

#downloading compressed 
download_data(granule_id_dry, use_case_folder ,username="esa",password="D25uXWjdjZEJ", compressed= True, compression_model="low" )
download_data(granule_id_flooded, use_case_folder ,username="esa",password="D25uXWjdjZEJ", compressed= True, compression_model="low" )

#downloading originals for comparison purpose
download_data(granule_id_dry, use_case_folder ,username="esa",password="D25uXWjdjZEJ", compressed= False)
download_data(granule_id_flooded, use_case_folder ,username="esa",password="D25uXWjdjZEJ", compressed= False)


output\flooding\S1A_IW_GRDH_1SDV_20210906T043536_20210906T043601_039556_04ACE3_18DD_12_compressed.hfc already downloaded.
output\flooding\S1A_IW_GRDH_1SDV_20211129T043536_20211129T043601_040781_04D736_2E2A_12_compressed.hfc already downloaded.
output\flooding\S1A_IW_GRDH_1SDV_20210906T043536_20210906T043601_039556_04ACE3_18DD_12.png already downloaded.
output\flooding\S1A_IW_GRDH_1SDV_20211129T043536_20211129T043601_040781_04D736_2E2A_12.png already downloaded.


('output\\flooding\\S1A_IW_GRDH_1SDV_20211129T043536_20211129T043601_040781_04D736_2E2A_12.png',
 0)

### decompressing (low)

In [23]:
!python ../high-fidelity-generative-compression-master/compress.py -i {use_case_folder} -ckpt {model_low_path} --save --output_dir {use_case_folder} --only_decompress True

getting data from  output\flooding
only_decompress is true... getting the .hfc
decompress working on device [ cuda ] 
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\src\loss\perceptual_similarity\weights\v0.1\alex.pth
...[net-lin [alex]] initialized
...Done
image [ output\flooding\S1A_IW_GRDH_1SDV_20210906T043536_20210906T043601_039556_04ACE3_18DD_12_compressed_RECON.png ] already decompressed, getting to the next
image [ output\flooding\S1A_IW_GRDH_1SDV_20211129T043536_20211129T043601_040781_04D736_2E2A_12_compressed_RECON.png ] already decompressed, getting to the next


15_53_12 INFO - logger_setup: D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\high-fidelity-generative-compression-master\compress.py

100%|##########| 64/64 [00:00<00:00, 271.33it/s]
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\VisioTerra\technique\P382_ESRIN_COPEX-DCC\engineering\HiFiC.git\HiFiC\pytorch_nightly_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
15_53_14 INFO - load_model

In [24]:
original_dry = os.path.join(use_case_folder,granule_id_dry+".png")
original_flooded = os.path.join(use_case_folder,granule_id_flooded+".png")
recon_dry = os.path.join(use_case_folder,granule_id_dry+"_compressed_RECON.png")
recon_flooded = os.path.join(use_case_folder,granule_id_flooded+"_compressed_RECON.png")

### before | after (originals)

In [25]:
display_widget = pn.Row(loading_gif, pn.pane.Str('Processing request...',styles={'font-size': '12pt'}))
display(display_widget) # loading screen while preparing next widget
#resizing width to get better viewing experience
img_width = 1500

ori_resized = resize_and_save(original_dry, width=img_width)
rec_resized = resize_and_save(original_flooded, width=img_width)
display_widget.clear();
#display_widget.append(pn.Swipe(recon_path_low, recon_path_high,slider_width=3, slider_color ="#00fff2"))
display_widget.append(pn.Swipe(ori_resized, rec_resized,slider_width=3, slider_color ="#00fff2"))

Row
    [0] GIF(str)
    [1] Str(str, styles={'font-size': '12pt'})

### before | after (reconstructed low)

In [26]:
display_widget = pn.Row(loading_gif, pn.pane.Str('Processing request...',styles={'font-size': '12pt'}))
display(display_widget) # loading screen while preparing next widget
#resizing width to get better viewing experience
img_width = 1500

ori_resized = resize_and_save(recon_dry, width=img_width)
rec_resized = resize_and_save(recon_flooded, width=img_width)
display_widget.clear();
#display_widget.append(pn.Swipe(recon_path_low, recon_path_high,slider_width=3, slider_color ="#00fff2"))
display_widget.append(pn.Swipe(ori_resized, rec_resized,slider_width=3, slider_color ="#00fff2"))

Row
    [0] GIF(str)
    [1] Str(str, styles={'font-size': '12pt'})